In [6]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install accelerate>=0.12.0
!pip install datasets
!pip install evaluate
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.8 MB/s eta 0:00:00


In [9]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/T5

Mounted at /content/drive
/content/drive/MyDrive/T5


In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import numpy as np

checkpoint = "T5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
new_words = ['{', '}']
tokenizer.add_tokens(new_words)

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint,return_dict=True)
model.resize_token_embeddings(len(tokenizer))

prefix = "translate English to Sparql: "
metric = evaluate.load("sacrebleu")


def preprocess_function(examples):
    inputs = [prefix + string["string"] for string in examples["question"]]
    targets = [ string["sparql"] for string in examples["query"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


data = load_dataset("orkg/SciQA")
print(data)
tokenized_books = data.map(preprocess_function, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="en2sparql_T5_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model()

Generating train split:   0%|          | 0/1795 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/513 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 1795
    })
    validation: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 257
    })
    test: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 513
    })
})


Map:   0%|          | 0/1795 [00:00<?, ? examples/s]

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

Map:   0%|          | 0/513 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,0.521352,0.001900,19.000000
2,No log,0.223593,0.002100,19.000000
3,1.346800,0.183063,0.003200,19.000000
4,1.346800,0.165879,0.003200,19.000000
5,0.201400,0.150620,0.003100,19.000000
6,0.201400,0.137010,0.003100,19.000000
7,0.151700,0.123802,0.003100,19.000000
8,0.151700,0.113055,0.003100,19.000000
9,0.116800,0.105649,0.003100,19.000000
10,0.116800,0.100053,0.003200,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1181: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1181: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1181: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1181: UserWarning: Using the model-agnostic default `max_length` (=20) to control